In [ ]:
import cdsapi
import xarray as xr
from urllib.request import urlopen
c = cdsapi.Client()
# dataset to read
dataset = 'reanalysis-era5-single-levels'
years = ['2017', '2018', '2019','2020', '2021', '2022']
for year in years:
    if year == '2017':
        months = ['03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ]
    elif year == '2022':
        months = ['01']
    else:
        months = ['01','02','03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ]
        
    for month in months:
        params = {'product_type': 'reanalysis',
                'variable': ['10m_u_component_of_wind', '10m_v_component_of_wind',],
                'year': year,
                'month': month,
                'day': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                    '13', '14', '15',
                    '16', '17', '18',
                    '19', '20', '21',
                    '22', '23', '24',
                    '25', '26', '27',
                    '28', '29', '30',
                    '31',
                ],
                'time': [
                    '00:00', '01:00', '02:00',
                    '03:00', '04:00', '05:00',
                    '06:00', '07:00', '08:00',
                    '09:00', '10:00', '11:00',
                    '12:00', '13:00', '14:00',
                    '15:00', '16:00', '17:00',
                    '18:00', '19:00', '20:00',
                    '21:00', '22:00', '23:00',
                ],
                'area': [38, -180, -38,180,],
                'format': 'netcdf',
            }
        # retrieves the path to the file
        fl = c.retrieve(dataset, params)
        with urlopen(fl.location) as f:
            ds = xr.open_dataset(f.read())

C:\Users\Syver\miniconda3\envs\pythonProject8\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2022-02-10 11:54:13,448 INFO Welcome to the CDS
2022-02-10 11:54:13,450 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
C:\Users\Syver\miniconda3\envs\pythonProject8\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2022-02-10 11:54:13,522 INFO Request is queued
C:\Users\Syver\miniconda3\envs\pythonProject8\lib\site-packages\urllib3\connectionpool